# Introduction to Random Forests

## Imports

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [3]:
from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

In [7]:
PATH = '../data/bulldozers/'

## Fast.ai Methods

In [19]:
!head ../data/bulldozers/Train.zip

'head' is not recognized as an internal or external command,
operable program or batch file.
